# Step 3: Recommender - Compute Relevance Between Requests and Articles
Attempts to determine how relevent each Content node is to each Request node and builds Recommendation nodes to store that score for highly rated relevance.

![step3](resources/Step3_Recommend.png)

The result of this step includes:
- Recommendation nodes, connected to Content nodes with a RECOMMENDS, and Request nodes with a RELATES_TO relationship

## Setup

In [ ]:
from opentldr import KnowledgeGraph
kg=KnowledgeGraph()

import opentldr.Domain as domain

## Parameters
OpenTLDR workflows use the notebook block tagged as "parameters" to inject variables (for example to change the recommendation thresholds).

> **Do Not Change Variable Names in the Parameters Block** you are welcome to change the values of these parameter variables, but please do not change their names. They are used elsewhere in the notebook and in other workflow processes.

In [ ]:
#Parameters
recommendation_threshold = 0.54


# Recommend Contents based on their relationships thru the KG to the Request
Relevance is scored to include:
- The distance of the node, thru the KG from the request
- With an integrated penelty for uncertainty in relationships
- The average relevance of a nodes neighbors

## Compute a score for closest nodes in KG based on distance from the query in the graph
- The decay_rate determines how much relevance goes down for each hop (default is 10%)

In [ ]:
decay_rate=0.1

def distance_decay(distance_scores:float, node_id:str, score:float):
    #print ("\nRecursing on node {id} with score of {score}.".format(id=node_id, score=score))
    node=kg.get_by_uid(node_id)
    #print ("\tNode is: {}".format(node.to_text()))
    
    # Avoid looking along paths that circumvent reference data
    nodetype=str(type(node))
    for avoid in ["Source", "User", "Recommendation", "Tldr", "Summary", "EvalKey"]:
        if avoid in nodetype:
            # print("\tAvoiding path through a {type} node.".format(type=nodetype))
            return

    prev_score = distance_scores.get(str(node_id),-999)
    if prev_score >= score:
        #print ("\tPreviously at node {id} with score of {prev} vs now with {current}.".format(id=node_id, prev=prev_score, current=score))
        return
    else:
        #print ("\tAssigning node {id} a score of {current}.".format(id=node_id,current=score))
        distance_scores[str(node_id)]= score
       
    # Since this is the lowest so far, we have to also check its neighbors    
    neighbor_records, meta = kg.neo4j_query("MATCH (h)<-[]->(n) WHERE h.uid='{uid}' RETURN n.uid".format(uid=node_id))
    for n_record in neighbor_records:
        other_id = n_record.data()["n.uid"]

        # recurse on nodes that are not too far already
        other_score = score - decay_rate
        if other_score > 0.0:
              distance_decay(distance_scores,other_id, round(other_score,2))

    return

### Average the distance relevance scores for each Article based on its neighbors 

In [ ]:
def average_content(distance_scores:dict):
    avg_scores=dict()
    all_content = kg.get_all_content()
    for article in all_content:
        acc=distance_scores.get(str(article.uid),0.0);
        count=1
        if acc > 0.0:
            for e in kg.get_entities_by_content(article):
                acc+=distance_scores.get(str(e.uid),0.0)
                count+=1
        avg_scores[str(article.uid)]=round(acc/count,3)
    return avg_scores

# Process Each Query in the System

In [ ]:
# if you plan to run this notebook repeatedly, you can clear out just the recommendations and redo them
# kg.delete_all_recommendations()

In [ ]:
from opentldr.Domain import Recommendation, Request, Content

all_requests = kg.get_all_requests()

for request in all_requests:
    distance_scores=dict()
    distance_decay(distance_scores,request.uid,1.0)
    content_scores=average_content(distance_scores)

    print("Request: ("+request.title+"): "+request.text)
    sorted_recommendations=sorted(content_scores.items(), key=lambda x:x[1], reverse=True)
    for art_item in sorted_recommendations:
        recommendation_score=art_item[1]
        content = kg.get_content_by_uid(art_item[0])

        if recommendation_score > recommendation_threshold:
            print("\tRecommending:\t"+ content.title +" ("+str(recommendation_score)+") ")
            kg.add_recommendation(request=request,content=content,score=recommendation_score)
        else:
            print("\tNOT Recommending:\t"+ content.title +" ("+str(recommendation_score)+") ")

    print()

In [ ]:
kg.close()